In [1]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env

In [2]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

df = preprocess(pd.read_pickle('./data/binance-ETHUSD-1h.pkl'))
df.head(5)

                       open    high     low   close       volume  \
date_open                                                          
2020-08-18 07:00:00  430.00  435.00  410.00  430.30   487.154463   
2020-08-18 08:00:00  430.27  431.79  430.27  430.80   454.176153   
2020-08-18 09:00:00  430.86  431.13  428.71  429.35  1183.710884   
2020-08-18 10:00:00  429.75  432.69  428.59  431.90  1686.183227   
2020-08-18 11:00:00  432.09  432.89  426.99  427.45  1980.692724   

                             date_close  
date_open                                
2020-08-18 07:00:00 2020-08-18 08:00:00  
2020-08-18 08:00:00 2020-08-18 09:00:00  
2020-08-18 09:00:00 2020-08-18 10:00:00  
2020-08-18 10:00:00 2020-08-18 11:00:00  
2020-08-18 11:00:00 2020-08-18 12:00:00  

In [3]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()

    df['feature_close'] = (df['close'] - df['close'].mean()) / df['close'].std()

    return df

df = preprocess(pd.read_pickle('./data/binance-ETHUSD-1h.pkl'))
df.head(5)

                       open    high     low   close       volume  \
date_open                                                          
2020-08-18 07:00:00  430.00  435.00  410.00  430.30   487.154463   
2020-08-18 08:00:00  430.27  431.79  430.27  430.80   454.176153   
2020-08-18 09:00:00  430.86  431.13  428.71  429.35  1183.710884   
2020-08-18 10:00:00  429.75  432.69  428.59  431.90  1686.183227   
2020-08-18 11:00:00  432.09  432.89  426.99  427.45  1980.692724   

                             date_close  feature_close  
date_open                                               
2020-08-18 07:00:00 2020-08-18 08:00:00      -1.891634  
2020-08-18 08:00:00 2020-08-18 09:00:00      -1.891128  
2020-08-18 09:00:00 2020-08-18 10:00:00      -1.892594  
2020-08-18 10:00:00 2020-08-18 11:00:00      -1.890016  
2020-08-18 11:00:00 2020-08-18 12:00:00      -1.894514  

In [4]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

obs, _ = env.reset()
# On veut une position de 88% ETH / 12% USD
obs, reward, terminated, truncated, info = env.step(0.88)
print(obs)
print(info)

In [5]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    position_range=(0, 1),  # ICI : (borne min, borne max)
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

In [6]:
from gym_trading_env.wrapper import DiscreteActionsWrapper

# Vous pouvez aussi appeler le wrapper `env` pour faire plus simple
# Ici, je fais explicitement la distinction entre `wrapper` et `env`
wrapper = DiscreteActionsWrapper(env, positions=[-1, 0, 0.25, 0.5, 0.75, 1, 2])
obs, _ = wrapper.reset()
# On veut une position de 25% ETH / 75% USD ; cela correspond à la position
# d'index 2 dans la liste ci-dessus
obs, reward, terminated, truncated, info = wrapper.step(2)
print(obs)
print(info)

In [7]:
def reward_function(history):
    return history['portfolio_valuation', -1]

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    # On spécifie la fonction de récompense
    reward_function=reward_function,
)

In [8]:
nb_episodes = 2
for episode in range(1, nb_episodes + 1):
    obs, _ = env.reset()
    print(f'Episode n˚{episode} -- Jeu de donnée {env.name}')
    done = False

    while not done:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    if terminated:
        print('Argent perdu')
    elif truncated:
        print('Épisode terminé')

In [9]:
def metric_portfolio_valuation(history):
    return round(history['portfolio_valuation', -1], 2)

env.add_metric('Portfolio Valuation', metric_portfolio_valuation)

done = False
obs, _ = env.reset()

while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

In [10]:
portfolio_valuation = env.historical_info['portfolio_valuation', -1]
# Si on avait WandB :
# run.summary['portfolio_valuation'] = portfolio_valuation
# On simule ça par un simple print...
print(portfolio_valuation)

In [11]:
metrics = env.get_metrics()
print(metrics)
portfolio_valuation = metrics['Portfolio Valuation']
print(portfolio_valuation)

In [12]:
def preprocess_v2(df):
    df = df.sort_index().dropna().drop_duplicates()

    # --- 1. Log Returns (Rendements Logarithmiques) ---
    df["feature_log_returns"] = np.log(df["close"]).diff()

    # --- 2. Indicateurs de Volatilité (ATR simplifié) ---
    df['tr1'] = df['high'] - df['low']
    df['tr2'] = np.abs(df['high'] - df['close'].shift(1))
    df['tr3'] = np.abs(df['low'] - df['close'].shift(1))
    df['tr'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)
    df['feature_atr'] = df['tr'].rolling(window=14).mean() / df["close"]

    # --- 3. Indicateurs de Tendance (MACD) ---
    ema_fast = df['close'].ewm(span=12, adjust=False).mean()
    ema_slow = df['close'].ewm(span=26, adjust=False).mean()
    df['feature_macd'] = ema_fast - ema_slow
    df['feature_macd_signal'] = df['feature_macd'].ewm(span=9, adjust=False).mean()

    # --- 4. Indicateurs de Momentum (RSI) ---
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['feature_rsi'] = 100 - (100 / (1 + rs)) / 100

    # --- 5. Nettoyage et Normalisation ---
    df = df.dropna()
    cols_to_normalize = ['feature_log_returns', 'feature_macd', 'feature_macd_signal', 'feature_atr']
    for col in cols_to_normalize:
        if df[col].std() > 0:
            df[col] = (df[col] - df[col].mean()) / df[col].std()
        else:
             df[col] = 0.0

    return df

In [13]:
def reward_function_v2(history):
    # Rendement log du portefeuille à l'étape t
    prev_val = history['portfolio_valuation', -2]
    curr_val = history['portfolio_valuation', -1]

    # Gestion du cas initial
    if prev_val == 0: return 0

    # Calcul du rendement log
    reward = np.log(curr_val / prev_val)

    # BONUS : Pénalité de risque (Sharpe Ratio simplifié)
    # Si vous voulez un agent prudent, vous pouvez soustraire une fraction de la volatilité récente
    return reward

In [14]:
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO # PPO avec LSTM
from stable_baselines3.common.vec_env import DummyVecEnv

# ... (insérer ici la fonction preprocess améliorée définie plus haut) ...
# ... (insérer ici la fonction reward_function définie plus haut) ...

# Création de l'environnement
# On utilise DummyVecEnv pour la compatibilité avec Stable Baselines 3
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess_v2,
    reward_function=reward_function_v2,
    position_range=(-1, 1),
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    # LIGNE SUPPRIMÉE : window_size=1
)

# Wrapper pour vectoriser (requis par SB3)
env = DummyVecEnv([lambda: env])

# Configuration du modèle
model = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=128,
    ent_coef=0.01,
    tensorboard_log="./tensorboard_logs/"
)

# Entrainement
print("Début de l'entraînement...")
model.learn(total_timesteps=100_000)
print("Entraînement terminé.")

In [15]:
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Récupération des fonctions (Doivent être identiques à l'entraînement) ---
# Copiez-collez ici votre fonction preprocess() améliorée de l'étape précédente
# et votre reward_function (bien que pour le test, la reward ne serve à rien,
# l'env en a besoin pour s'initialiser).

# --- 2. Chargement de l'environnement de Test ---
# Idéalement, pointez vers un fichier .pkl que l'agent n'a JAMAIS vu (ex: 2024.pkl)
# Si vous n'avez pas de données séparées, utilisez le même dossier mais gardez en tête
# que le résultat sera biaisé (overfitting).
env_test = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess_v2,
    reward_function=reward_function_v2,
    position_range=(-1, 1),
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    # CETTE LIGNE DOIT ÊTRE SUPPRIMÉE :
    # window_size=1
)

# On wrap l'environnement comme pour l'entraînement
env_test = DummyVecEnv([lambda: env_test])

# --- 3. Chargement de l'Agent ---
# On charge le modèle sauvegardé
model = RecurrentPPO.load("mon_agent_trading")

print("Modèle chargé avec succès. Début du backtest...")

In [16]:
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv

# --- NOUVELLES IMPORTATIONS ---
import wandb
from wandb.integration.sb3 import WandbCallback

# --- HYPERPARAMÈTRES (pour le suivi WandB) ---
config = {
    "policy_type": "MlpLstmPolicy",
    "total_timesteps": 100_000,
    "env_id": "MultiDatasetTradingEnv",
    "learning_rate": 3e-4,
    "n_steps": 2048,
    "batch_size": 128,
    "ent_coef": 0.01,
    # Ajoutez ici tous les paramètres de l'environnement (frais, capital, etc.)
}

# --- 1. INITIALISATION DE WANDB ---
run = wandb.init(
    project="RL-Trading-Project", # Nom de votre projet
    entity="arthur-collignon-cpe-lyon", # Remplacez par votre nom d'utilisateur WandB
    config=config,
    sync_tensorboard=True, # Synchroniser TensorBoard (si vous l'utilisez encore)
    monitor_gym=True,
    save_code=True,
)

# --- 2. CRÉATION DE L'ENVIRONNEMENT ET DU MODÈLE (Comme avant) ---

# ... (Vos fonctions preprocess/reward_function ici) ...

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    reward_function=reward_function,
    position_range=(-1, 1),
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

env = DummyVecEnv([lambda: env])

model = RecurrentPPO(
    config["policy_type"], # Utiliser le dictionnaire de config
    env,
    verbose=0, # Mettre à 0 pour éviter les logs console en faveur de WandB
    learning_rate=config["learning_rate"],
    n_steps=config["n_steps"],
    batch_size=config["batch_size"],
    ent_coef=config["ent_coef"],
    tensorboard_log=f"runs/{run.id}", # Pointer le log TensorBoard vers le dossier de WandB
)

# --- 3. DÉFINITION DU CALLBACK WANDB ---
wandb_callback = WandbCallback(
    model_save_path=f"models/{run.id}",
    verbose=1,
    model_save_freq=10000, # Sauvegarder le modèle tous les 10,000 pas
)


# --- 4. ENTRAÎNEMENT AVEC LE CALLBACK ---
try:
    print("Début de l'entraînement avec WandB...")
    model.learn(
        total_timesteps=config["total_timesteps"],
        callback=wandb_callback, # Passage du Callback ici
    )
finally:
    # --- 5. FIN DU RUN WANDB ---
    run.finish()

In [17]:
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Récupération des fonctions (Doivent être identiques à l'entraînement) ---
# Copiez-collez ici votre fonction preprocess() améliorée de l'étape précédente
# et votre reward_function (bien que pour le test, la reward ne serve à rien,
# l'env en a besoin pour s'initialiser).

# --- 2. Chargement de l'environnement de Test ---
# Idéalement, pointez vers un fichier .pkl que l'agent n'a JAMAIS vu (ex: 2024.pkl)
# Si vous n'avez pas de données séparées, utilisez le même dossier mais gardez en tête
# que le résultat sera biaisé (overfitting).
env_test = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess_v2,
    reward_function=reward_function_v2,
    position_range=(-1, 1),
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    # CETTE LIGNE DOIT ÊTRE SUPPRIMÉE :
    # window_size=1
)

# On wrap l'environnement comme pour l'entraînement
env_test = DummyVecEnv([lambda: env_test])

# --- 3. Chargement de l'Agent ---
# On charge le modèle sauvegardé
model = RecurrentPPO.load("mon_agent_trading")

print("Modèle chargé avec succès. Début du backtest...")

In [18]:
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv

# --- NOUVELLES IMPORTATIONS ---
import wandb
from wandb.integration.sb3 import WandbCallback

# --- HYPERPARAMÈTRES (pour le suivi WandB) ---
config = {
    "policy_type": "MlpLstmPolicy",
    "total_timesteps": 100_000,
    "env_id": "MultiDatasetTradingEnv",
    "learning_rate": 3e-4,
    "n_steps": 2048,
    "batch_size": 128,
    "ent_coef": 0.01,
    # Ajoutez ici tous les paramètres de l'environnement (frais, capital, etc.)
}

# --- 1. INITIALISATION DE WANDB ---
run = wandb.init(
    project="RL-Trading-Project", # Nom de votre projet
    entity="arthur-collignon-cpe-lyon", # Remplacez par votre nom d'utilisateur WandB
    config=config,
    sync_tensorboard=True, # Synchroniser TensorBoard (si vous l'utilisez encore)
    monitor_gym=True,
    save_code=True,
)

# --- 2. CRÉATION DE L'ENVIRONNEMENT ET DU MODÈLE (Comme avant) ---

# ... (Vos fonctions preprocess/reward_function ici) ...

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    reward_function=reward_function,
    position_range=(-1, 1),
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

env = DummyVecEnv([lambda: env])

model = RecurrentPPO(
    config["policy_type"], # Utiliser le dictionnaire de config
    env,
    verbose=0, # Mettre à 0 pour éviter les logs console en faveur de WandB
    learning_rate=config["learning_rate"],
    n_steps=config["n_steps"],
    batch_size=config["batch_size"],
    ent_coef=config["ent_coef"],
    tensorboard_log=f"runs/{run.id}", # Pointer le log TensorBoard vers le dossier de WandB
)

# --- DÉFINITION DE LA LISTE DE CALLBACKS ---
callback = CallbackList([
    # Callback SB3 WandB: gère les logs d'entraînement (reward, loss, entropie)
    WandbCallback(
        model_save_path=f"models/{run.id}",
        verbose=0,
        model_save_freq=10000,
    ),
    # Callback Personnalisé: gère les logs financiers (portfolio_valuation, returns)
    CustomTradingCallback(verbose=0), 
])


# --- ENTRAÎNEMENT AVEC LA LISTE DE CALLBACKS ---
try:
    print("Début de l'entraînement avec WandB...")
    model.learn(
        total_timesteps=config["total_timesteps"],
        callback=callback, # Passe la liste des deux Callbacks
    )
finally:
    run.finish()

In [19]:
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
import numpy as np
import pandas as pd

class CustomTradingCallback(BaseCallback):
    """
    Callback pour enregistrer les métriques financières finales dans WandB
    à la fin de chaque épisode.
    """
    def __init__(self, verbose: int = 0):
        super().__init__(verbose)
        self.episode_num = 0

    def _on_step(self) -> bool:
        # Vérifie si l'épisode est terminé
        if self.locals['dones'][0]: # L'indice 0 est pour notre DummyVecEnv simple
            self.episode_num += 1

            # 1. Accéder à l'environnement non-vectorisé
            # Le VecEnv est un wrapper, on doit aller chercher l'env "nu"
            raw_env = self.training_env.envs[0].unwrapped

            # 2. Calculer les métriques
            # Note: gym-trading-env calcule déjà les métriques finales
            metrics = raw_env.get_metrics()

            # --- Journalisation dans WandB ---

            # A. Le critère d'évaluation final (Portfolio Valuation)
            final_val = metrics.get('Portfolio Valuation')

            # B. Performance vs. Marché (pour le contexte)
            market_return_str = metrics.get('Market Return', '0.00%').strip()
            market_return = float(market_return_str.strip('%')) / 100

            portfolio_return_str = metrics.get('Portfolio Return', '0.00%').strip()
            portfolio_return = float(portfolio_return_str.strip('%')) / 100


            if self.logger is not None:
                # Enregistrer les métriques spécifiques
                self.logger.record("episode/final_portfolio_valuation", final_val)
                self.logger.record("episode/return_vs_market_pct", (portfolio_return - market_return) * 100)
                self.logger.record("episode/total_portfolio_return_pct", portfolio_return * 100)
                self.logger.record("episode/market_return_pct", market_return * 100)
                self.logger.record("episode/steps", raw_env.steps)

                # S'assurer que le log est écrit immédiatement
                self.logger.dump(step=self.num_timesteps)

            # Optionnel: Réinitialiser l'environnement si nécessaire (déjà fait par SB3)
        return True

In [20]:
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv

# --- NOUVELLES IMPORTATIONS ---
import wandb
from wandb.integration.sb3 import WandbCallback

# --- HYPERPARAMÈTRES (pour le suivi WandB) ---
config = {
    "policy_type": "MlpLstmPolicy",
    "total_timesteps": 100_000,
    "env_id": "MultiDatasetTradingEnv",
    "learning_rate": 3e-4,
    "n_steps": 2048,
    "batch_size": 128,
    "ent_coef": 0.01,
    # Ajoutez ici tous les paramètres de l'environnement (frais, capital, etc.)
}

# --- 1. INITIALISATION DE WANDB ---
run = wandb.init(
    project="RL-Trading-Project", # Nom de votre projet
    entity="arthur-collignon-cpe-lyon", # Remplacez par votre nom d'utilisateur WandB
    config=config,
    sync_tensorboard=True, # Synchroniser TensorBoard (si vous l'utilisez encore)
    monitor_gym=True,
    save_code=True,
)

# --- 2. CRÉATION DE L'ENVIRONNEMENT ET DU MODÈLE (Comme avant) ---

# ... (Vos fonctions preprocess/reward_function ici) ...

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    reward_function=reward_function,
    position_range=(-1, 1),
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

env = DummyVecEnv([lambda: env])

model = RecurrentPPO(
    config["policy_type"], # Utiliser le dictionnaire de config
    env,
    verbose=0, # Mettre à 0 pour éviter les logs console en faveur de WandB
    learning_rate=config["learning_rate"],
    n_steps=config["n_steps"],
    batch_size=config["batch_size"],
    ent_coef=config["ent_coef"],
    tensorboard_log=f"runs/{run.id}", # Pointer le log TensorBoard vers le dossier de WandB
)

# --- DÉFINITION DE LA LISTE DE CALLBACKS ---
callback = CallbackList([
    # Callback SB3 WandB: gère les logs d'entraînement (reward, loss, entropie)
    WandbCallback(
        model_save_path=f"models/{run.id}",
        verbose=0,
        model_save_freq=10000,
    ),
    # Callback Personnalisé: gère les logs financiers (portfolio_valuation, returns)
    CustomTradingCallback(verbose=0), 
])


# --- ENTRAÎNEMENT AVEC LA LISTE DE CALLBACKS ---
try:
    print("Début de l'entraînement avec WandB...")
    model.learn(
        total_timesteps=config["total_timesteps"],
        callback=callback, # Passe la liste des deux Callbacks
    )
finally:
    run.finish()

In [21]:
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
import numpy as np
import pandas as pd

class CustomTradingCallback(BaseCallback):
    """
    Callback pour enregistrer les métriques financières finales dans WandB
    à la fin de chaque épisode.
    """
    def __init__(self, verbose: int = 0):
        super().__init__(verbose)
        self.episode_num = 0

    def _on_step(self) -> bool:
        # Vérifie si l'épisode est terminé
        if self.locals['dones'][0]:
            self.episode_num += 1

            raw_env = self.training_env.envs[0].unwrapped
            metrics = raw_env.get_metrics()

            # 1. Accéder à l'environnement non-vectorisé
            # Le VecEnv est un wrapper, on doit aller chercher l'env "nu"
            raw_env = self.training_env.envs[0].unwrapped

            # 2. Calculer les métriques
            # Note: gym-trading-env calcule déjà les métriques finales
            metrics = raw_env.get_metrics()

            # --- Journalisation dans WandB ---

            # A. Le critère d'évaluation final (Portfolio Valuation)
            final_val = metrics.get('Portfolio Valuation')

            # B. Performance vs. Marché (pour le contexte)
            market_return_str = metrics.get('Market Return', '0.00%').strip()
            market_return = float(market_return_str.strip('%')) / 100

            portfolio_return_str = metrics.get('Portfolio Return', '0.00%').strip()
            portfolio_return = float(portfolio_return_str.strip('%')) / 100


            if self.logger is not None:
                # Enregistrer les métriques spécifiques
                self.logger.record("episode/final_portfolio_valuation", final_val)
                self.logger.record("episode/return_vs_market_pct", (portfolio_return - market_return) * 100)
                self.logger.record("episode/total_portfolio_return_pct", portfolio_return * 100)
                self.logger.record("episode/market_return_pct", market_return * 100)
                # CORRECTION ICI : Utiliser raw_env.step au lieu de raw_env.steps
                self.logger.record("episode/steps", raw_env.step)
                
                # S'assurer que le log est écrit immédiatement
                self.logger.dump(step=self.num_timesteps)

        return True

In [22]:
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv

# --- NOUVELLES IMPORTATIONS ---
import wandb
from wandb.integration.sb3 import WandbCallback

# --- HYPERPARAMÈTRES (pour le suivi WandB) ---
config = {
    "policy_type": "MlpLstmPolicy",
    "total_timesteps": 100_000,
    "env_id": "MultiDatasetTradingEnv",
    "learning_rate": 3e-4,
    "n_steps": 2048,
    "batch_size": 128,
    "ent_coef": 0.01,
    # Ajoutez ici tous les paramètres de l'environnement (frais, capital, etc.)
}

# --- 1. INITIALISATION DE WANDB ---
run = wandb.init(
    project="RL-Trading-Project", # Nom de votre projet
    entity="arthur-collignon-cpe-lyon", # Remplacez par votre nom d'utilisateur WandB
    config=config,
    sync_tensorboard=True, # Synchroniser TensorBoard (si vous l'utilisez encore)
    monitor_gym=True,
    save_code=True,
)

# --- 2. CRÉATION DE L'ENVIRONNEMENT ET DU MODÈLE (Comme avant) ---

# ... (Vos fonctions preprocess/reward_function ici) ...

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    reward_function=reward_function,
    position_range=(-1, 1),
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

env = DummyVecEnv([lambda: env])

model = RecurrentPPO(
    config["policy_type"], # Utiliser le dictionnaire de config
    env,
    verbose=0, # Mettre à 0 pour éviter les logs console en faveur de WandB
    learning_rate=config["learning_rate"],
    n_steps=config["n_steps"],
    batch_size=config["batch_size"],
    ent_coef=config["ent_coef"],
    tensorboard_log=f"runs/{run.id}", # Pointer le log TensorBoard vers le dossier de WandB
)

# --- DÉFINITION DE LA LISTE DE CALLBACKS ---
callback = CallbackList([
    # Callback SB3 WandB: gère les logs d'entraînement (reward, loss, entropie)
    WandbCallback(
        model_save_path=f"models/{run.id}",
        verbose=0,
        model_save_freq=10000,
    ),
    # Callback Personnalisé: gère les logs financiers (portfolio_valuation, returns)
    CustomTradingCallback(verbose=0),
])


# --- ENTRAÎNEMENT AVEC LA LISTE DE CALLBACKS ---
try:
    print("Début de l'entraînement avec WandB...")
    model.learn(
        total_timesteps=config["total_timesteps"],
        callback=callback, # Passe la liste des deux Callbacks
    )
finally:
    run.finish()